In [38]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, random
import cv2
from glob import glob
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score, classification_report
from pathlib import Path
from PIL import Image 

model = tf.keras.models.load_model('modelkedua.h5')

# preprocess image
def preprocess(image):
    # convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # blur the image to reduce noise
    blur = cv2.GaussianBlur(gray, (3, 3), 6)
    # apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)
    return thresh


def main_outline(contours):
    main = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                main = approx
                max_area = area
    return main, max_area

def reframe(points):
    points = points.reshape((4, 2))
    new_points = np.zeros((4, 1, 2), dtype=np.int32)
    add = points.sum(1)
    new_points[0] = points[np.argmin(add)]
    new_points[3] = points[np.argmax(add)]
    diff = np.diff(points, axis=1)
    new_points[1] = points[np.argmin(diff)]
    new_points[2] = points[np.argmax(diff)]
    return new_points

def splitcells(image):
    rows = np.vsplit(image, 9)
    cells = []
    for r in rows:
        cols = np.hsplit(r, 9)
        for cell in cols:
            cells.append(cell)
    return cells


def CropCell(cells):
    cropped_cells = []
    for image in cells:
        img = np.array(image)
        img = img[4:46, 4:46]
        img = Image.fromarray(img)
        cropped_cells.append(img)
    return cropped_cells

def read_cells(cell, model):
    sudoku = []
    for i in cell:
        img = np.array(i)
        img = img[4:img.shape[0] - 4, 4:img.shape[1] - 4]
        img = cv2.resize(img, (32, 32))
        img = img / 255.0
        img = img.reshape(1, 32, 32, 1)
        prediction = model.predict(img)
        classIndex = np.argmax(prediction, axis=1)
        probabilityValue = np.amax(prediction)
        if probabilityValue > 0.65:
            sudoku.append(classIndex[0])
        else:
            sudoku.append(0)
    return sudoku


contour_1 = sudoku_a.copy()
contour_2 = sudoku_a.copy()

black_img = np.zeros((450, 450, 3), np.uint8)
main, max_area = main_outline(contours)
if main.size != 0:
    main = reframe(main)
    cv2.drawContours(black_img, [main], 0, (255, 255, 255), 2)
    cv2.drawContours(contour_1, [main], 0, (0, 255, 0), 2)
    pts1 = np.float32(main)
    pts2 = np.float32([[0, 0], [450, 0], [0, 450], [450, 450]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    img_warp_colored = cv2.warpPerspective(sudoku_a, matrix, (450, 450))
    img_warp_colored = cv2.cvtColor(img_warp_colored, cv2.COLOR_BGR2GRAY)




def cek(img):
    grid = read_cells(img, model)
    grid = np.asarray(grid)
    grid = grid.reshape(9, 9)
    return grid

def next_box(quiz):
    for i in range(9):
        for j in range(9):
            if quiz[i][j] == 0:
                return i, j
    return False

def possible(y, x, n, quiz):
    for i in range(9):
        if quiz[y][i] == n and x != i:
            return False
    for i in range(9):
        if quiz[i][x] == n and y != i:
            return False
    x0 = (x // 3) * 3
    y0 = (y // 3) * 3
    for i in range(3):
        for j in range(3):
            if quiz[y0 + i][x0 + j] == n and (y0 + i, x0 + j) != (y, x):
                return False
    return True

def solve(quiz):
    find = next_box(quiz)
    if not find:
        return True
    else:
        y, x = find
    for i in range(1, 10):
        if possible(y, x, i, quiz):
            quiz[y][x] = i
            if solve(quiz):
                return True
            quiz[y][x] = 0
    return False

def print_sudoku(quiz):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("---------------------")
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print(" | ", end="")
            if j == 8:
                print(quiz[i][j])
            else:
                print(str(quiz[i][j]) + " ", end="")
    print("---------------------")


sudoku_a = cv2.imread('/Volumes/DATAMAC/notebook/Sudoku/Sudoku-Hero/datasetsudoku/image1005.jpg')
sudoku_a = cv2.resize(sudoku_a, (450, 450))
thresh = preprocess(sudoku_a)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cv2.drawContours(contour_1, contours, -1, (0, 255, 0), 3)
sudoku_cell = splitcells(img_warp_colored)
sudoku_cell_croped = CropCell(sudoku_cell)
# grid = read_cells(sudoku_cell_croped, model)
# grid = np.asanyarray(grid)
# grid = grid.reshape(9, 9)
# print(grid)

def sudoku(img):
    sudoku_a = cv2.imread(img)
    sudoku_a = cv2.resize(sudoku_a, (450, 450))
    thresh = preprocess(sudoku_a)

    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # cv2.drawContours(contour_1, contours, -1, (0, 255, 0), 3)
    sudoku_cell = splitcells(img_warp_colored)
    sudoku_cell_croped = CropCell(sudoku_cell)
    grid = read_cells(sudoku_cell_croped, model)
    grid = np.asanyarray(grid)
    grid = grid.reshape(9, 9)
    return grid

def solve_sudoku(img):
    grid = sudoku(img)
    solve(grid)
    # hasil = print_sudoku(grid)
    return grid

sudoku('/Volumes/DATAMAC/notebook/Sudoku/Sudoku-Hero/datasetsudoku/image1005.jpg')

1/1 [==============================] - 0s 37ms/step


array([[0, 8, 0, 0, 0, 7, 0, 0, 9],
       [1, 0, 0, 0, 0, 0, 6, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 8, 0],
       [0, 0, 2, 0, 3, 0, 0, 0, 7],
       [0, 0, 0, 2, 1, 4, 0, 0, 0],
       [5, 0, 0, 0, 9, 0, 4, 0, 0],
       [0, 5, 0, 0, 0, 3, 0, 0, 0],
       [0, 0, 4, 0, 0, 0, 0, 0, 3],
       [6, 0, 0, 1, 0, 0, 0, 2, 0]])

In [41]:
sudoku('/Volumes/DATAMAC/notebook/Sudoku/Sudoku-Hero/datasetsudoku/image1005.jpg')

1/1 [==============================] - 0s 59ms/step


array([[0, 8, 0, 0, 0, 7, 0, 0, 9],
       [1, 0, 0, 0, 0, 0, 6, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 8, 0],
       [0, 0, 2, 0, 3, 0, 0, 0, 7],
       [0, 0, 0, 2, 1, 4, 0, 0, 0],
       [5, 0, 0, 0, 9, 0, 4, 0, 0],
       [0, 5, 0, 0, 0, 3, 0, 0, 0],
       [0, 0, 4, 0, 0, 0, 0, 0, 3],
       [6, 0, 0, 1, 0, 0, 0, 2, 0]])

In [42]:
print(grid)

[[4 8 5 6 2 7 3 1 9]
 [1 2 3 4 8 9 6 7 5]
 [7 9 6 3 5 1 2 8 4]
 [8 4 2 5 3 6 1 9 7]
 [3 7 9 2 1 4 8 5 6]
 [5 6 1 7 9 8 4 3 2]
 [2 5 8 9 6 3 7 4 1]
 [9 1 4 8 7 2 5 6 3]
 [6 3 7 1 4 5 9 2 8]]


In [28]:
cek(sudoku_cell_croped)

1/1 [==============================] - 0s 38ms/step


array([[0, 8, 0, 0, 0, 7, 0, 0, 9],
       [1, 0, 0, 0, 0, 0, 6, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 8, 0],
       [0, 0, 2, 0, 3, 0, 0, 0, 7],
       [0, 0, 0, 2, 1, 4, 0, 0, 0],
       [5, 0, 0, 0, 9, 0, 4, 0, 0],
       [0, 5, 0, 0, 0, 3, 0, 0, 0],
       [0, 0, 4, 0, 0, 0, 0, 0, 3],
       [6, 0, 0, 1, 0, 0, 0, 2, 0]])

In [25]:
# grid = read_cells(sudoku_cell_croped, model)
grid = np.asarray(grid)
grid = grid.reshape(9, 9)
print(grid)

[[0 8 0 0 0 7 0 0 9]
 [1 0 0 0 0 0 6 0 0]
 [0 0 0 3 0 0 0 8 0]
 [0 0 2 0 3 0 0 0 7]
 [0 0 0 2 1 4 0 0 0]
 [5 0 0 0 9 0 4 0 0]
 [0 5 0 0 0 3 0 0 0]
 [0 0 4 0 0 0 0 0 3]
 [6 0 0 1 0 0 0 2 0]]
